In [105]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from api_keys import gmaps_key

In [106]:
# Import clean car data
car_data_df = pd.read_csv("../DataSource/clean_car_data.csv")
car_data_df.head()

,Unnamed: 0,VIN (1-10),County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Legislative District,DOL Vehicle ID,Vehicle Location,2020 Census Tract
0,1,5YJXCBE27J,Island,Greenbank,WA,98253,2018,TESLA,MODEL X,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,238,10.0,326880081,(-122.575569 48.08489),5.302997e+10
1,8,1N4BZ0CP8H,Chelan,Wenatchee,WA,98801,2017,NISSAN,LEAF,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,107,12.0,192768747,(-120.336499 47.425146),5.300796e+10
2,10,WVWKR7AU3K,Snohomish,Edmonds,WA,98026,2019,VOLKSWAGEN,E-GOLF,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,125,32.0,477565112,(-122.333046 47.829439),5.306105e+10
3,11,5YJ3E1EB0J,Island,Clinton,WA,98236,2018,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,215,10.0,476402678,(-122.35803 47.9796),5.302997e+10
4,13,5YJ3E1EBXL,Snohomish,Brier,WA,98036,2020,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,322,1.0,103288153,(-122.281856 47.811678),5.306105e+10


In [107]:
# Create a new dataframe to hold only the data that I plan to use
reduced_car_data_df = car_data_df[["City", "Vehicle Location"]]
reduced_car_data_df.head()

,City,Vehicle Location
0,Greenbank,(-122.575569 48.08489)
1,Wenatchee,(-120.336499 47.425146)
2,Edmonds,(-122.333046 47.829439)
3,Clinton,(-122.35803 47.9796)
4,Brier,(-122.281856 47.811678)


In [108]:
# Groupby Location (count)
heat_data_df = reduced_car_data_df.groupby(reduced_car_data_df["Vehicle Location"], as_index=False).count()
heat_data_df.head()


,Vehicle Location,City
0,(-102.69968 22.95716),7
1,(-117.06882 47.7251),20
2,(-117.07351 47.22679),3
3,(-117.0768 46.9101),9
4,(-117.083224 47.660645),114


In [116]:
# Reduce to significant data (>1500 for test, use ~200 on Actual)
filtered_heat_data_df = heat_data_df.loc[heat_data_df['City'] > 5]
filtered_heat_data_df.head()

,Vehicle Location,City
0,(-102.69968 22.95716),7
1,(-117.06882 47.7251),20
3,(-117.0768 46.9101),9
4,(-117.083224 47.660645),114
5,(-117.11195 47.70763),31


In [117]:
# Extract Vehicle Location into useable numbers

# First, split the column by ' ', join the resultant columns to the rest, and rename them as Lng and Lat
complete_heat_data_df = pd.merge(filtered_heat_data_df, filtered_heat_data_df["Vehicle Location"].str.split(' ', expand=True), how="left", left_index=True, right_index=True)
complete_heat_data_df = complete_heat_data_df.rename(columns={1: 'Lng', 2: 'Lat'})

# Remove '(' and ')' from the columns
complete_heat_data_df['Lat'] = complete_heat_data_df['Lat'].replace({r"\)":''}, regex=True)
complete_heat_data_df['Lng'] = complete_heat_data_df['Lng'].replace({r"\(":''}, regex=True)

# Cast as numbers
complete_heat_data_df['Lat'] = complete_heat_data_df['Lat'].astype('float')
complete_heat_data_df['Lng'] = complete_heat_data_df['Lng'].astype('float')

# Display current
complete_heat_data_df.head()

,Vehicle Location,City,0,Lng,Lat
0,(-102.69968 22.95716),7,,-102.699680,22.957160
1,(-117.06882 47.7251),20,,-117.068820,47.725100
3,(-117.0768 46.9101),9,,-117.076800,46.910100
4,(-117.083224 47.660645),114,,-117.083224,47.660645
5,(-117.11195 47.70763),31,,-117.111950,47.707630


In [118]:
# Configure the map plot
gmaps.configure(api_key= gmaps_key)
heatMapBase = gmaps.figure(center= (47.5, -120), zoom_level= 7, layout={
        'width': '1200px',
        'height': '800px',
        'padding': '3px',
        'border': '1px solid black'
})

# Add the heatmap layer
heatMapLayer = gmaps.heatmap_layer(complete_heat_data_df[["Lat", "Lng"]], weights= complete_heat_data_df["City"], dissipating= False, max_intensity= 750, point_radius= 0.1)
heatMapBase.add_layer(heatMapLayer)

# Display the map
heatMapBase

Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='3px', width='1200px'))